In [ ]:
import math
from collections import deque

# Volumes of jugs
VA = math.sqrt(5.0)
VB = math.sqrt(3.0)
VC = math.sqrt(2.0)

# For convenience, keep them in a list. We'll index jugs as 0=A, 1=B, 2=C.
CAP = [VA, VB, VC]

# We'll check if a volume is "close enough" to 1.
def is_goal(volume):
    return 0.9997 <= volume <= 1.0003

# Rounding function so that floating states do not explode combinatorially.
# Use 5 decimal places as a compromise.
def rounded_state(a, b, c):
    return (round(a, 5), round(b, 5), round(c, 5))

def pour(src_val, dst_val, src_cap, dst_cap):
    """
    Pour from 'src' jug to 'dst' jug until src is empty or dst is full.
    Returns (new_src_val, new_dst_val).
    """
    # Available in src
    available = src_val
    # Remaining capacity in dst
    space = dst_cap - dst_val

    if available <= space:
        # We can pour everything from src to dst
        new_dst = dst_val + available
        new_src = 0.0
    else:
        # We fill dst to the brim
        new_dst = dst_cap
        new_src = available - space

    return new_src, new_dst

def bfs_solution():
    """
    Run a BFS to find the shortest sequence of steps that leads to
    one jug having volume in [0.9997, 1.0003].
    Return (num_steps, [moves]) if found, else None.
    """

    # We'll store a queue of tuples: (a, b, c, path)
    # 'path' is a list of two-letter moves, e.g. "TA", "AB", etc.
    start_state = (0.0, 0.0, 0.0)  # All jugs initially empty
    start_r = rounded_state(*start_state)

    # BFS queue
    queue = deque()
    queue.append((start_r[0], start_r[1], start_r[2], []))

    visited = set()
    visited.add(start_r)

    while queue:
        a, b, c, path = queue.popleft()

        # Check goal condition
        if is_goal(a) or is_goal(b) or is_goal(c):
            # Found a solution!
            return (len(path), path)

        # Generate successors
        current_state = (a, b, c)

        # 1. Fill from tap: T->A, T->B, T->C
        fill_ops = [
            ("TA", (VA, b, c)),
            ("TB", (a, VB, c)),
            ("TC", (a, b, VC))
        ]

        # 2. Empty into sink: A->S, B->S, C->S
        empty_ops = [
            ("AS", (0.0, b, c)),
            ("BS", (a, 0.0, c)),
            ("CS", (a, b, 0.0))
        ]

        # 3. Cross pours: A->B, B->A, A->C, C->A, B->C, C->B
        # We'll define them via the 'pour' helper.
        # Each operation yields a new (a', b', c').
        cross_ops = []

        # A->B
        na, nb = pour(a, b, VA, VB)
        cross_ops.append(("AB", (na, nb, c)))

        # B->A
        nb, na = pour(b, a, VB, VA)
        cross_ops.append(("BA", (na, nb, c)))

        # A->C
        na, nc = pour(a, c, VA, VC)
        cross_ops.append(("AC", (na, b, nc)))

        # C->A
        nc, na = pour(c, a, VC, VA)
        cross_ops.append(("CA", (na, b, nc)))

        # B->C
        nb, nc = pour(b, c, VB, VC)
        cross_ops.append(("BC", (a, nb, nc)))

        # C->B
        nc, nb = pour(c, b, VC, VB)
        cross_ops.append(("CB", (a, nb, nc)))

        all_ops = fill_ops + empty_ops + cross_ops

        for move_label, (aa, bb, cc) in all_ops:
            rstate = rounded_state(aa, bb, cc)
            if rstate not in visited:
                visited.add(rstate)
                queue.append((rstate[0], rstate[1], rstate[2],
                              path + [move_label]))

    # If we exhaust the queue without finding a solution, return None
    return None

def main():
    result = bfs_solution()
    if result is None:
        print("No solution found (within the step limit).")
    else:
        steps, moves = result
        print(steps)
        print(" ".join(moves))

if __name__ == "__main__":
    main()

39
TC CA TC CA AS CA TC CA TC AB CA TC CA AS CA TC CA AS CA TC CA TC CA AS CA TC CA TC CA AS CA TC CA AS CA TC CA TC CA
